# LangGraph Agent Deployment to Amazon Bedrock AgentCore

This notebook demonstrates how to deploy a FastAPI LangGraph agent to Amazon Bedrock AgentCore Runtime and invoke it.

It's tested to run in the Python virtual environment created in `cx-agent-backend`

## Prerequisites

1. AWS CLI configured with appropriate permissions
2. Docker installed with ARM64 support
3. ECR repository created
4. Agent runtime IAM role created

In [ ]:
import boto3
import uuid
import os

## Configuration

In [ ]:
# Configuration
AWS_REGION = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-east-1"))
AWS_ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
ECR_REPOSITORY = "langgraph-cx-agent"
IMAGE_TAG = "latest"
ECR_URI = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{ECR_REPOSITORY}:{IMAGE_TAG}"
AGENT_RUNTIME_NAME = "langgraph_cx_agent"
AGENT_RUNTIME_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/agentic-ai-bedrock-role" # deployed in the prerequisite section
USER_POOL_ID = ""  # TODO: Enter Cognito User Pool ID from the terraform deployment output
CLIENT_ID = ""  # TODO: Enter Cognito App Client ID from the terraform deployment output
CLIENT_SECRET = ""  # TODO: Enter Cognito App Client Secret from the AWS console

print(f"Account ID: {AWS_ACCOUNT_ID}")
print(f"Region: {AWS_REGION}")
print(f"ECR URI: {ECR_URI}")
print(f"User Pool ID: {USER_POOL_ID}")

if not (USER_POOL_ID and CLIENT_ID and CLIENT_SECRET):
    raise ValueError(
        "Please set USER_POOL_ID, CLIENT_ID, and CLIENT_SECRET above"
    )


In [ ]:
# Create ECR repository to store your container image
import boto3

ecr_client = boto3.client('ecr', region_name=AWS_REGION)

try:
    response = ecr_client.create_repository(repositoryName=ECR_REPOSITORY)
    print(f"Repository created: {response['repository']['repositoryUri']}")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print("Repository already exists")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Login to ECR
!aws ecr get-login-password --region {AWS_REGION} | docker login --username AWS --password-stdin {AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com

# Alternatively with Finch:
#!aws ecr get-login-password --region {AWS_REGION} | finch login --username AWS --password-stdin {AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com

In [ ]:
# Build and tag Docker image for ARM64
!cd cx-agent-backend && docker buildx build --platform linux/arm64 -t {ECR_URI} --push .

# Alternatively with Finch:
#!cd cx-agent-backend && finch build --platform linux/arm64 -t {ECR_URI} . && finch push {ECR_URI}

## Step 2: Create Agent Runtime

In [ ]:
# Create Bedrock AgentCore client
agentcore_client = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)

# Create agent runtime
try:
    response = agentcore_client.create_agent_runtime(
        agentRuntimeName=AGENT_RUNTIME_NAME,
        agentRuntimeArtifact={
            'containerConfiguration': {
                'containerUri': ECR_URI
            }
        },
        networkConfiguration={"networkMode": "PUBLIC"},
        roleArn=AGENT_RUNTIME_ROLE_ARN,
        authorizerConfiguration={
        'customJWTAuthorizer': {
            'discoveryUrl': f'https://cognito-idp.{AWS_REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration',
            'allowedClients': [
                CLIENT_ID
            ]
        }
    }
    )
    
    agent_runtime_arn = response['agentRuntimeArn']
    print(f"Agent runtime created: {agent_runtime_arn}")
    
except Exception as e:
    print(f"Error creating agent runtime: {e}")
    # If already exists, get the ARN
    try:
        response = agentcore_client.get_agent_runtime(agentRuntimeName=AGENT_RUNTIME_NAME)
        agent_runtime_arn = response['agentRuntimeArn']
        print(f"Using existing agent runtime: {agent_runtime_arn}")
    except Exception as e2:
        print(f"Error getting agent runtime: {e2}")
        raise

## Step 3: Retrieve Amazon Cognito Access Token for testing the agent

In [ ]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("EMAIL_USERNAME")
_set_if_undefined("TEMPORARY_PASSWORD")
_set_if_undefined("PERMANENT_PASSWORD")

In [ ]:
# Access values later in the notebook
username = os.environ['EMAIL_USERNAME']
temp_password = os.environ['TEMPORARY_PASSWORD']
permanent_password = os.environ['PERMANENT_PASSWORD']

In [ ]:
# Create test user
cognito_client = boto3.client('cognito-idp', region_name=AWS_REGION)

try:
    response = cognito_client.admin_create_user(
        UserPoolId=USER_POOL_ID,
        Username=username,
        TemporaryPassword=temp_password,
        MessageAction='SUPPRESS'
    )
    print(f"User created: {response['User']['Username']}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Set permanent password
try:
    response = cognito_client.admin_set_user_password(
        UserPoolId=USER_POOL_ID,
        Username=username,
        Password=permanent_password,
        Permanent=True
    )
    print("Password set successfully")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
# Calculate SECRET_HASH and store as variable
import hmac
import hashlib
import base64
import os

def calculate_secret_hash(username, client_id, client_secret):
    message = username + client_id
    return base64.b64encode(
        hmac.new(
            client_secret.encode('utf-8'),
            message.encode('utf-8'),
            hashlib.sha256
        ).digest()
    ).decode('utf-8')



SECRET_HASH = calculate_secret_hash(username, CLIENT_ID, CLIENT_SECRET)

In [ ]:
# Get access token
try:
    response = cognito_client.initiate_auth(
        ClientId=CLIENT_ID,
        AuthFlow='USER_PASSWORD_AUTH',
        AuthParameters={
            'USERNAME': username,
            'PASSWORD': permanent_password,
            'SECRET_HASH': SECRET_HASH
        }
    )
    
    access_token = response['AuthenticationResult']['AccessToken']
    os.environ["COGNITO_TOKEN"] = access_token
    print("Access token fetched")
    
except Exception as e:
    print(f"Error: {e}")

## Step 4: Invoke Agent Runtime

In [ ]:
import requests
import urllib.parse
import json

def invoke_agent(message, agent_arn, auth_token, session_id, region=AWS_REGION):
    """Invoke Bedrock AgentCore runtime with a message."""
    escaped_agent_arn = urllib.parse.quote(agent_arn, safe='')
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id
    }
    
    response = requests.post(url, headers=headers, data=json.dumps({"input": {"prompt": message, "conversation_id": "12345"}}), timeout=61)
    
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error ({response.status_code}): {response.text}")
        return None

## Step 5: Test Different Scenarios

In [ ]:
AUTH_TOKEN = os.environ["COGNITO_TOKEN"]  # (See above)
AGENT_ARN = agent_runtime_arn
SESSION_ID = str(uuid.uuid4())
    
result = invoke_agent("Hello, can you help me?", AGENT_ARN, AUTH_TOKEN, SESSION_ID)
if result:
    print(json.dumps(result, indent=2))

In [ ]:
# Test math functionality
math_test = "What is 25 + 17?"
print(f"Testing math with: {math_test}")
result = invoke_agent(math_test, AGENT_ARN, AUTH_TOKEN, SESSION_ID)
if result:
    print(json.dumps(result, indent=2))
print("\n" + "="*50 + "\n")


result = invoke_agent("Add 10 to the result", AGENT_ARN, AUTH_TOKEN, SESSION_ID)
if result:
    print(json.dumps(result, indent=2))
print("\n" + "="*50 + "\n")

## Step 6: Cleanup (Optional)

In [ ]:
# Uncomment to delete the agent runtime
# AGENT_RUNTIME_NAME = ""
# try:
#     agentcore_client.delete_agent_runtime(agentRuntimeId=AGENT_RUNTIME_NAME)
#     print(f"Agent runtime {AGENT_RUNTIME_NAME} deleted")
# except Exception as e:
#     print(f"Error deleting agent runtime: {e}")